# [Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk)
## Preprocessing - Bureau table
### Platform: Python 3, colab.research.google.com

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import seaborn as sns
import plotly
import plotly.graph_objs as go
from google.colab import drive

In [2]:
plotly.offline.init_notebook_mode(connected=True)
sns.set(style='whitegrid', context='notebook')

In [0]:
# run in each cell where Plotly is used
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

## Load data

In [4]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
bureau_orig = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/kaggle-home-credit-default-risk/data/bureau.csv", header=0)
bureau_orig.shape

(1716428, 17)

In [6]:
bureau_orig.head(5)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [7]:
bureau_orig.shape

(1716428, 17)

## Exploration

In [8]:
for i in range(len(bureau_orig.columns)):
    print(bureau_orig.columns[i], bureau_orig.dtypes[i])

SK_ID_CURR int64
SK_ID_BUREAU int64
CREDIT_ACTIVE object
CREDIT_CURRENCY object
DAYS_CREDIT int64
CREDIT_DAY_OVERDUE int64
DAYS_CREDIT_ENDDATE float64
DAYS_ENDDATE_FACT float64
AMT_CREDIT_MAX_OVERDUE float64
CNT_CREDIT_PROLONG int64
AMT_CREDIT_SUM float64
AMT_CREDIT_SUM_DEBT float64
AMT_CREDIT_SUM_LIMIT float64
AMT_CREDIT_SUM_OVERDUE float64
CREDIT_TYPE object
DAYS_CREDIT_UPDATE int64
AMT_ANNUITY float64


In [9]:
describe_numerical = bureau_orig.describe()
for i in describe_numerical.columns:
    print(i)
    print(describe_numerical.loc[:, i])
    print()

SK_ID_CURR
count    1.716428e+06
mean     2.782149e+05
std      1.029386e+05
min      1.000010e+05
25%      1.888668e+05
50%      2.780550e+05
75%      3.674260e+05
max      4.562550e+05
Name: SK_ID_CURR, dtype: float64

SK_ID_BUREAU
count    1.716428e+06
mean     5.924434e+06
std      5.322657e+05
min      5.000000e+06
25%      5.463954e+06
50%      5.926304e+06
75%      6.385681e+06
max      6.843457e+06
Name: SK_ID_BUREAU, dtype: float64

DAYS_CREDIT
count    1.716428e+06
mean    -1.142108e+03
std      7.951649e+02
min     -2.922000e+03
25%     -1.666000e+03
50%     -9.870000e+02
75%     -4.740000e+02
max      0.000000e+00
Name: DAYS_CREDIT, dtype: float64

CREDIT_DAY_OVERDUE
count    1.716428e+06
mean     8.181666e-01
std      3.654443e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.792000e+03
Name: CREDIT_DAY_OVERDUE, dtype: float64

DAYS_CREDIT_ENDDATE
count    1.610875e+06
mean     5.105174e+02
std      4.994220e+03
min     

Plot numerical features

In [10]:
def plot_numerical_feature(feature, min_x=None, max_x=None, size=None):
    configure_plotly_browser_state()
    if not min_x:
        min_x = describe_numerical.loc["min", feature]
    if not max_x:
        max_x = describe_numerical.loc["max", feature]
    if not size:
        size = int((max_x - min_x)/30)
    plotly.offline.iplot({
        "data": [go.Histogram(
            x=bureau_orig.loc[:, feature],
            xbins={
                "start": min_x,
                "end": max_x,
                "size": size
            }
        )],
        "layout": go.Layout(title="{} histogram".format(feature), height=300)
    })
plot_numerical_feature("AMT_CREDIT_SUM_DEBT", min_x=0.1, max_x=1000000)

Output hidden; open in https://colab.research.google.com to view.

Plot categorical features

In [11]:
def plot_categorical_feature(feature):
    configure_plotly_browser_state()
    feature_counts = bureau_orig[feature].value_counts()
    plotly.offline.iplot({
        "data": [go.Bar(
            x=feature_counts.index,
            y=feature_counts.values
        )],
        "layout": go.Layout(title="{} bar".format(feature), height=300)
    })
# plot_categorical_feature("CREDIT_CURRENCY")
plot_categorical_feature("CREDIT_ACTIVE")

In [0]:
bureau = bureau_orig.copy(deep=True) # retain original features for later

## Duplicates

In [13]:
bureau.loc[bureau.duplicated()==True, ]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY


## Missing data

In [14]:
def get_missing(df):
    missing = df.isnull().sum()
    missing = missing.sort_values(ascending=False)
    return missing.loc[missing > 0]
print("# of columns with missing values")
print(len(get_missing(bureau)))
get_missing(bureau)

# of columns with missing values
7


AMT_ANNUITY               1226791
AMT_CREDIT_MAX_OVERDUE    1124488
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_DEBT        257669
DAYS_CREDIT_ENDDATE        105553
AMT_CREDIT_SUM                 13
dtype: int64

Manual column drop:

In [15]:
print("Shape pre column drop: {}".format(bureau.shape))

bureau = bureau.drop("AMT_ANNUITY", axis=1)

print("Shape post column drop: {}".format(bureau.shape))
print("Remaining # of columns with missing values")
print(len(get_missing(bureau)))
get_missing(bureau)

Shape pre column drop: (1716428, 17)
Shape post column drop: (1716428, 16)
Remaining # of columns with missing values
6


AMT_CREDIT_MAX_OVERDUE    1124488
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_DEBT        257669
DAYS_CREDIT_ENDDATE        105553
AMT_CREDIT_SUM                 13
dtype: int64

Manual column fill:

In [16]:
print("Shape pre column drop: {}".format(bureau.shape))

fill_dict = {"AMT_CREDIT_MAX_OVERDUE": 0}
bureau = bureau.fillna(fill_dict)

print("Shape post column drop: {}".format(bureau.shape))
print("Remaining # of columns with missing values")
print(len(get_missing(bureau)))
get_missing(bureau)

Shape pre column drop: (1716428, 16)
Shape post column drop: (1716428, 16)
Remaining # of columns with missing values
5


DAYS_ENDDATE_FACT       633653
AMT_CREDIT_SUM_LIMIT    591780
AMT_CREDIT_SUM_DEBT     257669
DAYS_CREDIT_ENDDATE     105553
AMT_CREDIT_SUM              13
dtype: int64

Fill all types *object* using mode:

In [17]:
def fill_objects(df):
    fill_dict_object = {}
    for i in get_missing(df).index:
        if df.loc[:, i].dtype == "object":
            fill_dict_object[i] = df[i].mode()[0]
    df = df.fillna(fill_dict_object)
    return df
bureau = fill_objects(bureau)

print("Remaining # of columns with missing values")
print(len(get_missing(bureau)))
get_missing(bureau)

Remaining # of columns with missing values
5


DAYS_ENDDATE_FACT       633653
AMT_CREDIT_SUM_LIMIT    591780
AMT_CREDIT_SUM_DEBT     257669
DAYS_CREDIT_ENDDATE     105553
AMT_CREDIT_SUM              13
dtype: int64

Fill all numerical types using median:

In [18]:
fill_dict_numerical = {}
def fill_numerical(df):
    for i in get_missing(df).index:
        fill_dict_numerical[i] = df[i].median()
    df = df.fillna(fill_dict_numerical)
    return df

bureau = fill_numerical(bureau)
print("Remaining # of columns with missing values")
print(len(get_missing(bureau)))
assert 0 == len(get_missing(bureau))

Remaining # of columns with missing values
0


Shape after filing missing values:

In [19]:
bureau.shape

(1716428, 16)

## Outliers

In [20]:
descriptive_stats = bureau.describe()
for i in descriptive_stats.columns:
    mean = descriptive_stats.loc["mean", i]
    std = descriptive_stats.loc["std", i]
    outliers_high = bureau.loc[bureau[i] > mean + 10 * std, i]
    if len(outliers_high) > 0:
        print(i, len(outliers_high))

CREDIT_DAY_OVERDUE 832
AMT_CREDIT_MAX_OVERDUE 107
CNT_CREDIT_PROLONG 9114
AMT_CREDIT_SUM 1346
AMT_CREDIT_SUM_DEBT 3060
AMT_CREDIT_SUM_LIMIT 3660
AMT_CREDIT_SUM_OVERDUE 159


## Correlated features

In [21]:
def show_feature_correlation(df):
    df_corr = df.corr()
    high_correlations = pd.DataFrame(columns=["f1", "f2", "corr"])
    for i, row in df_corr.iterrows():
        for j in row.index:
            if i == j:
                continue
            exists = high_correlations.loc[
                ((high_correlations["f1"] == i) & (high_correlations["f2"] == j)) | 
                ((high_correlations["f2"] == i) & (high_correlations["f1"] == j))]
            if len(exists) > 0:
                continue           
            high_correlations.loc[len(high_correlations), :] = [i, j, abs(df_corr.loc[i, j])]
    high_correlations = high_correlations.sort_values(by="corr", ascending=False)
    return high_correlations
high_correlations = show_feature_correlation(bureau)
print(len(high_correlations))
print(high_correlations.head(10))

78
                     f1                   f2      corr
25          DAYS_CREDIT    DAYS_ENDDATE_FACT  0.710411
32          DAYS_CREDIT   DAYS_CREDIT_UPDATE  0.688771
56    DAYS_ENDDATE_FACT   DAYS_CREDIT_UPDATE  0.650971
68       AMT_CREDIT_SUM  AMT_CREDIT_SUM_DEBT  0.559133
49  DAYS_CREDIT_ENDDATE   DAYS_CREDIT_UPDATE  0.238897
24          DAYS_CREDIT  DAYS_CREDIT_ENDDATE  0.214802
42  DAYS_CREDIT_ENDDATE    DAYS_ENDDATE_FACT   0.14569
74  AMT_CREDIT_SUM_DEBT   DAYS_CREDIT_UPDATE  0.137713
29          DAYS_CREDIT  AMT_CREDIT_SUM_DEBT  0.129944
44  DAYS_CREDIT_ENDDATE   CNT_CREDIT_PROLONG  0.113808


In [22]:
feature1 = "DAYS_CREDIT"
feature2 = "DAYS_ENDDATE_FACT"
configure_plotly_browser_state()
random_idx = np.random.randint(low=0, high=len(bureau)-1, size=5000) # limit size to save memory
plotly.offline.iplot({
    "data": [go.Scatter(
        x=bureau.loc[random_idx, feature1],
        y=bureau.loc[random_idx, feature2],
        mode = "markers",
        marker = {
            "size": 2
        }
    )],
    "layout": go.Layout(title="{} - {} scatter".format(feature1, feature2), height=300)
})

## Skewed data

Features that may benefit from non-linear transformations

In [23]:
skewed_features = ["CREDIT_DAY_OVERDUE"]
# TODO explore other features
plot_numerical_feature(skewed_features[0], max_x=500)

Output hidden; open in https://colab.research.google.com to view.

## Imbalanced classes

Imbalanced features that may benefit from additional treatment:

In [24]:
imbalanced_classes = ["CREDIT_ACTIVE", "CREDIT_ACTIVE"]
# TODO explore other features
plot_categorical_feature(imbalanced_classes[0])

Features likely worth removing because a single class represents > 99%

In [25]:
imbalanced_classes_remove = ["CREDIT_CURRENCY"]
# TODO explore other features
plot_categorical_feature(imbalanced_classes_remove[0])

## Save datasets

In [0]:
bureau.to_csv("/content/gdrive/My Drive/Colab Notebooks/kaggle-home-credit-default-risk/data/bureau_clean.csv")

In [27]:
bureau_check = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/kaggle-home-credit-default-risk/data/bureau_clean.csv", header=0, index_col=0)
assert bureau.shape == bureau_check.shape

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [29]:
bureau.dtypes

SK_ID_CURR                  int64
SK_ID_BUREAU                int64
CREDIT_ACTIVE              object
CREDIT_CURRENCY            object
DAYS_CREDIT                 int64
CREDIT_DAY_OVERDUE          int64
DAYS_CREDIT_ENDDATE       float64
DAYS_ENDDATE_FACT         float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG          int64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE                object
DAYS_CREDIT_UPDATE          int64
dtype: object